In [ ]:
# This code block is importing various Python libraries and modules that are commonly used
# for data manipulation, analysis, and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import ConnectionPatch
import matplotlib.colors as mcolors


# DataAnalyzer is a class that set up a data analysis environment for working with booking and
# session data
class DataAnalyzer:
    # The __init__ method (constructor) It reads the bookings CSV file into a pandas DataFrame:
    def __init__(self, bookings_file, sessions_file):
        # bookings_file: is the file path to a CSV containing booking data
        self.df_bookings = pd.read_csv(bookings_file)
        # sessions_file: is file path to a CSV containing session data
        # This creates an instance variable df_sessions that contains the session data
        self.df_sessions = pd.read_csv(sessions_file)
        # The method merge_dataframes() and assigns the result to self.df_combined
        self.df_combined = self.merge_dataframes()

    # rIt uses pandas' merge function to combine self.df_bookings and self.df_sessions.

    # The merge is performed on the "booking_id" column, which is common to both DataFrames.
    # The how="outer" parameter specifies an outer join, meaning it will keep all rows
    # from both DataFrames, filling in NaN for missing values where necessary.
    def merge_dataframes(self):
        return pd.merge(
            self.df_bookings, self.df_sessions, on="booking_id", how="outer"
        )

    # this method analyzes the combined DataFrame to count sessions with single bookings
    # and multiple bookings:
    def get_sessions_booking_counts(self):
        # The groups the combined DataFrame by "session_id" and counts the
        # unique "booking_id" values for each session:
        session_booking_counts = self.df_combined.groupby("session_id")[
            "booking_id"
        ].nunique()
        single_booking = (session_booking_counts == 1).sum()
        multiple_bookings = (session_booking_counts > 1).sum()
        return {
            "Single Booking": single_booking,
            "Multiple Bookings": multiple_bookings,
        }


# Visualizer class you've provided is designed to create visualizations based on the data it receives
class Visualizer:
    # The class is initialized with a data parameter, which is stored as an instance variable.
    # This data is expected to be used for creating visualizations.
    def __init__(self, data):
        self.data = data

    # The plot_pie_and_bar method: This method is responsible for creating a figure with two subplots
    # side by side - a pie chart and a bar chart.  It creates a figure with two subplots arranged
    # horizontally (1 row, 2 columns) with a total size of 15x7 inches.
    def plot_pie_and_bar(self):
        # Create a pie chart on the left subplot ( ax1) using the labels, sizes, colors, and
        # explode data.  Create a bar chart on the right subplot ( ax2) using the same labels
        # and sizes.
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
        # This code snippet is preparing data and parameters for creating a pie chart
        # This line is creating a list of labels from the keys of the self.data dictionary.
        # These labels will be used to identify each slice of the pie chart
        labels = list(self.data.keys())
        # This line is creating a list of sizes from the values of the self.data dictionary
        sizes = list(self.data.values())
        # This line is defining a list of two colors to be used in the pie chart.
        # "#ff9999" is a light red color, and "#66b3ff" is a light blue color.
        colors = ["#ff9999", "#66b3ff"]
        explode = (0, 0.1)

        # Pie chart
        ax1.pie(
            sizes,
            explode=explode,
            labels=labels,
            colors=colors,
            autopct="%1.1f%%",
            startangle=90,
        )
        ax1.axis("equal")
        ax1.set_title("Sessions with Single vs Multiple Bookings")

        # Bar chart
        x = range(len(labels))
        ax2.bar(x, sizes, color=colors)
        ax2.set_ylabel("Number of Sessions")
        ax2.set_title("Sessions with Single vs Multiple Bookings")
        ax2.set_xticks(x)
        ax2.set_xticklabels(labels)

        # Add labels on top of bars
        for i, v in enumerate(sizes):
            ax2.text(i, v, str(v), ha="center", va="bottom")

        # Add connection patch
        for i, (start, end) in enumerate(zip(ax1.patches, ax2.patches)):
            ith = start.theta1 + (start.theta2 - start.theta1) / 2
            jth = end.get_x() + end.get_width() / 2
            con = ConnectionPatch(
                xyA=(np.cos(ith), np.sin(ith)),
                coordsA=ax1.transData,
                xyB=(jth, 0),
                coordsB=ax2.transData,
                color=colors[i],
                linewidth=1,
                arrowstyle="-",
            )
            ax2.add_artist(con)

        plt.tight_layout()
        plt.show()

    def plot_3d_bar(self):
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection="3d")

        labels = list(self.data.keys())
        sizes = list(self.data.values())
        x = np.arange(len(labels))

        dx = dy = 0.5
        dz = sizes

        colors = ["#ff9999", "#66b3ff"]
        color_map = [mcolors.to_rgba(c) for c in colors]

        for i, (x_pos, height) in enumerate(zip(x, dz)):
            ax.bar3d(x_pos, 0, 0, dx, dy, height, color=color_map[i], shade=True)

        ax.set_xticks(x + dx / 2)
        ax.set_xticklabels(labels)
        ax.set_ylabel("Category")
        ax.set_zlabel("Number of Sessions")
        ax.set_title("Sessions with Single vs Multiple Bookings (3D)")

        # Add a legend
        legend_elements = [
            plt.Rectangle((0, 0), 1, 1, facecolor=c, edgecolor="none") for c in colors
        ]
        ax.legend(legend_elements, labels, loc="upper right")

        plt.tight_layout()
        plt.show()


# This code defines the main() function, which serves as the entry point for the program
def main():
    # This creates an instance of the DataAnalyzer class, initializing it with two CSV files:
    # "Bookings.csv" and "Sessions.csv".
    analyzer = DataAnalyzer("Bookings.csv", "Sessions.csv")
    session_data = analyzer.get_sessions_booking_counts()
    # Print the session booking counts:
    print("Session Booking Counts:")
    # This loop prints out the category Single Booking, Multiple Bookings count.
    for category, count in session_data.items():
        print(f"{category}: {count}")

    print(
        f"\nNumber of Sessions with more than one booking: {session_data['Multiple Bookings']}"
    )

    visualizer = Visualizer(session_data)
    visualizer.plot_pie_and_bar()
    visualizer.plot_3d_bar()


if __name__ == "__main__":
    main()